In [30]:
import pandas as pd

In [31]:
train = pd.read_csv('Match_dataset.csv')
result = pd.read_csv('match_results.csv')

In [32]:
train.info()

<class 'pandas.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Match_ID               600 non-null    str    
 1   Date                   600 non-null    str    
 2   Venue                  600 non-null    str    
 3   Team_A                 600 non-null    str    
 4   Team_B                 600 non-null    str    
 5   Stage                  600 non-null    str    
 6   Team_A_Ranking         600 non-null    int64  
 7   Team_B_Ranking         600 non-null    int64  
 8   Team_A_Form            600 non-null    float64
 9   Team_B_Form            600 non-null    float64
 10  HeadToHead_A_Wins      600 non-null    int64  
 11  HeadToHead_B_Wins      600 non-null    int64  
 12  Venue_HomeAdvantage_A  600 non-null    int64  
 13  Venue_HomeAdvantage_B  600 non-null    int64  
 14  Pitch_Type             600 non-null    str    
 15  Avg_T20_Score_Ven

In [33]:
result.info()

<class 'pandas.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   Match_ID  600 non-null    str  
 1   Winner    600 non-null    str  
dtypes: str(2)
memory usage: 9.5 KB


In [34]:
train.head()

,Match_ID,Date,Venue,Team_A,Team_B,Stage,Team_A_Ranking,Team_B_Ranking,Team_A_Form,Team_B_Form,HeadToHead_A_Wins,HeadToHead_B_Wins,Venue_HomeAdvantage_A,Venue_HomeAdvantage_B,Pitch_Type,Avg_T20_Score_Venue,Toss_Winner,Toss_Decision,Team_A_Tech_Index,Team_B_Tech_Index,Match_Total
0,M0001,2026-03-06,Chennai,New Zealand,Scotland,Super8,5,13,76.97,54.62,6,4,0,0,Spin-Friendly,155,Team_A,Field,251.27,156.37,168
1,M0002,2026-03-07,Mumbai,South Africa,Sri Lanka,Final,6,9,64.19,58.75,13,6,0,0,Flat,175,Team_B,Bat,249.33,187.44,150
2,M0003,2026-03-02,Kolkata,India,PNG,Group,1,20,69.87,45.14,13,0,1,0,Spin-Friendly,165,Team_B,Field,298.24,54.03,160
3,M0004,2026-02-22,Colombo,Netherlands,Ireland,Group,11,12,85.59,61.94,10,4,0,0,Spin-Friendly,160,Team_A,Bat,197.19,174.95,145
4,M0005,2026-02-28,Mumbai,Namibia,South Africa,Group,15,6,45.98,88.52,3,15,0,0,Flat,175,Team_B,Bat,124.91,271.68,185


In [35]:
cols = ['Venue','Team_A','Team_B','Stage','Pitch_Type','Toss_Winner','Toss_Decision']

for col in cols:
    print(train[col].unique())

<StringArray>
['Chennai', 'Mumbai', 'Kolkata', 'Colombo', 'Ahmedabad', 'Delhi', 'Kandy']
Length: 7, dtype: str
<StringArray>
[ 'New Zealand', 'South Africa',        'India',  'Netherlands',
      'Namibia',  'Afghanistan',          'USA',         'Oman',
       'Canada',     'Pakistan',      'Ireland',        'Nepal',
    'Sri Lanka',          'PNG',     'Scotland',    'Australia',
     'Zimbabwe',      'England',  'West Indies',   'Bangladesh']
Length: 20, dtype: str
<StringArray>
[    'Scotland',    'Sri Lanka',          'PNG',      'Ireland',
 'South Africa',          'USA',        'Nepal',  'Netherlands',
  'West Indies',       'Canada',   'Bangladesh',      'England',
     'Zimbabwe',         'Oman',     'Pakistan',        'India',
  'New Zealand',      'Namibia',    'Australia',  'Afghanistan']
Length: 20, dtype: str
<StringArray>
['Super8', 'Final', 'Group', 'SemiFinal']
Length: 4, dtype: str
<StringArray>
['Spin-Friendly', 'Flat', 'Pace-Friendly']
Length: 3, dtype: str
<StringA

In [36]:
train.drop(columns=['Match_ID','Date'])

,Venue,Team_A,Team_B,Stage,Team_A_Ranking,Team_B_Ranking,Team_A_Form,Team_B_Form,HeadToHead_A_Wins,HeadToHead_B_Wins,Venue_HomeAdvantage_A,Venue_HomeAdvantage_B,Pitch_Type,Avg_T20_Score_Venue,Toss_Winner,Toss_Decision,Team_A_Tech_Index,Team_B_Tech_Index,Match_Total
0,Chennai,New Zealand,Scotland,Super8,5,13,76.97,54.62,6,4,0,0,Spin-Friendly,155,Team_A,Field,251.27,156.37,168
1,Mumbai,South Africa,Sri Lanka,Final,6,9,64.19,58.75,13,6,0,0,Flat,175,Team_B,Bat,249.33,187.44,150
2,Kolkata,India,PNG,Group,1,20,69.87,45.14,13,0,1,0,Spin-Friendly,165,Team_B,Field,298.24,54.03,160
3,Colombo,Netherlands,Ireland,Group,11,12,85.59,61.94,10,4,0,0,Spin-Friendly,160,Team_A,Bat,197.19,174.95,145
4,Mumbai,Namibia,South Africa,Group,15,6,45.98,88.52,3,15,0,0,Flat,175,Team_B,Bat,124.91,271.68,185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,Kolkata,England,Zimbabwe,Group,3,14,77.24,61.10,19,3,0,0,Spin-Friendly,165,Team_B,Field,294.94,154.29,164
596,Colombo,Australia,England,Group,2,3,85.50,78.79,8,8,0,0,Spin-Friendly,160,Team_B,Bat,296.33,278.65,187
597,Chennai,Canada,Australia,Group,18,2,36.86,72.24,4,23,0,0,Spin-Friendly,155,Team_A,Bat,70.96,285.08,149
598,Delhi,Namibia,Pakistan,Group,15,4,48.37,53.93,1,7,0,0,Pace-Friendly,170,Team_B,Bat,129.25,253.03,197


In [37]:
train.drop(columns=['Match_ID','Date'],inplace=True)

In [38]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(drop='first',sparse_output=False)

encoded = ohe.fit_transform(train[cols])

encoded_df = pd.DataFrame(
    encoded,
    columns=ohe.get_feature_names_out(cols)
)

train = pd.concat([train.drop(columns=cols),encoded_df], axis=1)

In [39]:
train.info()

<class 'pandas.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 63 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Team_A_Ranking            600 non-null    int64  
 1   Team_B_Ranking            600 non-null    int64  
 2   Team_A_Form               600 non-null    float64
 3   Team_B_Form               600 non-null    float64
 4   HeadToHead_A_Wins         600 non-null    int64  
 5   HeadToHead_B_Wins         600 non-null    int64  
 6   Venue_HomeAdvantage_A     600 non-null    int64  
 7   Venue_HomeAdvantage_B     600 non-null    int64  
 8   Avg_T20_Score_Venue       600 non-null    int64  
 9   Team_A_Tech_Index         600 non-null    float64
 10  Team_B_Tech_Index         600 non-null    float64
 11  Match_Total               600 non-null    int64  
 12  Venue_Chennai             600 non-null    float64
 13  Venue_Colombo             600 non-null    float64
 14  Venue_Delhi          

In [40]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

result['Winner'] = le.fit_transform(result['Winner'])

In [41]:
result.head()

,Match_ID,Winner
0,M0001,0
1,M0002,0
2,M0003,0
3,M0004,0
4,M0005,1


In [42]:
X = train.drop(columns=['Match_Total'])
y = result['Winner']

In [43]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler



X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LogisticRegression()

model.fit(X_train_scaled,y_train)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

In [44]:
preds = model.predict(X_test_scaled)

In [45]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(preds,y_test)

In [46]:
print(accuracy)

0.85


In [47]:
import joblib

joblib.dump(model, 't20_wc_26.pkl')
joblib.dump(le, 'label_encoder.pkl')
joblib.dump(ohe, 'onehot_encoder.pkl')

['onehot_encoder.pkl']

In [48]:
from sklearn.metrics import f1_score

f1 = f1_score(y_test,preds, average='weighted')
print(f1)

0.8518970968669157


In [53]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test,preds)

array([[62, 13],
       [ 5, 40]])

In [55]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression

categorical_cols = [...]
numeric_cols = [...]

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols)
    ]
)

model = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("clf", LogisticRegression(
            max_iter=1000,
            solver="lbfgs"
        ))
    ]
)


In [56]:
from sklearn.model_selection import cross_val_score

cv_f1 = cross_val_score(
    model,
    X,
    y,
    cv=5,
    scoring="f1_weighted"
)

print("CV F1 mean:", cv_f1.mean())
print("CV F1 std:", cv_f1.std())


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "d:\Codes\ML_Practice\env_3.12\Lib\site-packages\sklearn\model_selection\_validation.py", line 833, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\Codes\ML_Practice\env_3.12\Lib\site-packages\sklearn\base.py", line 1336, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Codes\ML_Practice\env_3.12\Lib\site-packages\sklearn\pipeline.py", line 613, in fit
    Xt = self._fit(X, y, routed_params, raw_params=params)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Codes\ML_Practice\env_3.12\Lib\site-packages\sklearn\p